In [1]:
import pandas as pd
import numpy as np
import json
import h5py
import gzip
import os
from tqdm.notebook import tqdm

In [2]:
from selfpeptide.utils.processing_utils import extract_peptides_from_protein_sequence
from selfpeptide.utils.constants import *
from selfpeptide.utils.trie import Trie

In [3]:
hdf5_file = '../processed_data/peptide_reference_dataset.hdf5'

# Create Dataset

In [4]:
with h5py.File(hdf5_file, 'w') as f:
    print(f.keys())

<KeysViewHDF5 []>


## Save reference human proteome

In [5]:
with open("../processed_data/UniProt/human_peptides.json", "r") as f:
    reference_human_peptides = json.load(f)
reference_human_peptides = set(reference_human_peptides) 
len(reference_human_peptides)

52208587

In [6]:
with h5py.File(hdf5_file, 'r+') as f:
    hmn_grp = f.create_group("human_proteome")
    dset1 = hmn_grp.create_dataset('reference_human_proteome', data=list(reference_human_peptides), compression='gzip', chunks=True)

In [9]:
with h5py.File(hdf5_file, 'r+') as f:
    print(len(f["human_proteome"]['reference_human_proteome'])) 
    print(f["human_proteome"]['reference_human_proteome'][:10])        

52208587
[b'CYDPTLDVW' b'TNITLVLCDSCN' b'GICDFDKGDGSH' b'GRIRDPIL' b'SSSLHCASSICS'
 b'HTICYFTEFL' b'PPMLSRGLGH' b'SSLEGFQW' b'VDANLQKLTQ' b'LLAINQRFQ']


## Save cancer peptides

In [14]:
n_other_peptides = 0

In [15]:
with open("../processed_data/XMAn_filtered_peptides.json", "r") as f:
    filtered_cancer_peptides_set = json.load(f)
n_other_peptides += len(filtered_cancer_peptides_set)
len(filtered_cancer_peptides_set)

39702919

In [11]:
with h5py.File(hdf5_file, 'r+') as f:
    hmn_grp = f['human_proteome']
    dset2 = hmn_grp.create_dataset('human_cancer_peptides', data=filtered_cancer_peptides_set, compression='gzip', chunks=True)

## Process bacterial proteomes

In [ ]:
# bacterial_peptides = set()
# # collected_bacterial_peptides = set()


# for fname in tqdm(os.listdir("../data/UniProt/bacterial_proteomes")):
#     if not fname.endswith("fasta.gz"):
#         continue
        
    
#     with gzip.open("../data/UniProt/bacterial_proteomes/{}".format(fname), "r") as f:
#         bacterial_proteins = []
#         prot_seq = ""
#         # for line in tqdm(f):
#         for line in f:
#             ln = line.decode()
#             if ln.startswith(">"):
#                 if len(prot_seq)>0:
#                     bacterial_proteins.append(prot_seq)
#                     prot_seq = ""
#                 continue
#             else:
#                 prot_seq += ln.strip()
#         bacterial_proteins.append(prot_seq)
        
#         species_peptides = []
#         for l in range(MIN_PEPTIDE_LEN, MAX_PEPTIDE_LEN+1):
#             for prot_seq in bacterial_proteins:
#                 peptides = extract_peptides_from_protein_sequence(prot_seq, l)
#                 species_peptides.extend(peptides)
#         species_peptides = [p for p in species_peptides if p not in reference_human_peptides]
#         bacterial_peptides.update(species_peptides)
#         # print(len(bacterial_peptides))
#         if len(bacterial_peptides)>1e7:
#             break
# len(bacterial_peptides)

In [ ]:
# with open(hdf5_file, 'r+') as f:
#     dset3 = f.create_dataset('bacterial_peptides', data=list(bacterial_peptides))

In [16]:
# bacterial_peptides = set()
# collected_bacterial_peptides = set()

with h5py.File(hdf5_file, 'r+') as f:
    # bact_grp = f.create_group("bacterial_proteomes")
    for fname in tqdm(os.listdir("../data/UniProt/bacterial_proteomes")):
        if not fname.endswith("fasta.gz"):
            continue
        proteome_id = fname.split(".")[0]
        
        with gzip.open("../data/UniProt/bacterial_proteomes/{}".format(fname), "r") as f:
            bacterial_proteins = []
            prot_seq = ""
            # for line in tqdm(f):
            for line in f:
                ln = line.decode()
                if ln.startswith(">"):
                    if len(prot_seq)>0:
                        bacterial_proteins.append(prot_seq)
                        prot_seq = ""
                    continue
                else:
                    prot_seq += ln.strip()
            bacterial_proteins.append(prot_seq)

        species_peptides = []
        for l in range(MIN_PEPTIDE_LEN, MAX_PEPTIDE_LEN+1):
            for prot_seq in bacterial_proteins:
                peptides = extract_peptides_from_protein_sequence(prot_seq, l)
                species_peptides.extend(peptides)
        species_peptides = [p for p in species_peptides if p not in reference_human_peptides]
        species_peptides = list(set(species_peptides))
        # dset = bact_grp.create_dataset(proteome_id, data=species_peptides, chunks=True)
        print("Proteome {}, n.peptides = {}".format(proteome_id, len(species_peptides)))
        n_other_peptides += len(species_peptides)

  0%|          | 0/218 [00:00<?, ?it/s]

Proteome UP000295518, n.peptides = 1381524
Proteome UP000032414, n.peptides = 4754652
Proteome UP000199512, n.peptides = 2888875
Proteome UP000502118, n.peptides = 1125634
Proteome UP000054608, n.peptides = 4619014
Proteome UP000500686, n.peptides = 1155611
Proteome UP000069902, n.peptides = 4072763
Proteome UP000500947, n.peptides = 739007
Proteome UP000284892, n.peptides = 4409673
Proteome UP000289440, n.peptides = 1344065
Proteome UP000252477, n.peptides = 1078787
Proteome UP000013568, n.peptides = 2485028
Proteome UP000000586, n.peptides = 2791461
Proteome UP000254634, n.peptides = 2641697
Proteome UP000055035, n.peptides = 4475800
Proteome UP000008205, n.peptides = 6774899
Proteome UP000007841, n.peptides = 7895756
Proteome UP000319776, n.peptides = 990965
Proteome UP000000747, n.peptides = 7094689
Proteome UP000201169, n.peptides = 2512824
Proteome UP000187359, n.peptides = 8069952
Proteome UP000054698, n.peptides = 4945170
Proteome UP000001522, n.peptides = 2214115
Proteome UP00

In [15]:
fname

'UP000295518.fasta.gz'

## Process viral proteomes

In [5]:
# viral_peptides = set()
# # collected_bacterial_peptides = set()


# for fname in tqdm(os.listdir("../data/UniProt/viral_proteomes")):
#     if not fname.endswith("fasta.gz"):
#         continue
        
    
#     with gzip.open("../data/UniProt/viral_proteomes/{}".format(fname), "r") as f:
#         viral_proteins = []
#         prot_seq = ""
#         # for line in tqdm(f):
#         for line in f:
#             ln = line.decode()
#             if ln.startswith(">"):
#                 if len(prot_seq)>0:
#                     viral_proteins.append(prot_seq)
#                     prot_seq = ""
#                 continue
#             else:
#                 prot_seq += ln.strip()
#         viral_proteins.append(prot_seq)
        
#         species_peptides = []
#         for l in range(MIN_PEPTIDE_LEN, MAX_PEPTIDE_LEN+1):
#             for prot_seq in viral_proteins:
#                 peptides = extract_peptides_from_protein_sequence(prot_seq, l)
#                 species_peptides.extend(peptides)
#         species_peptides = [p for p in species_peptides if p not in reference_human_peptides]
#         viral_peptides.update(species_peptides)
#         # print(len(bacterial_peptides))
#         if len(viral_peptides)>1e7:
#             break
# len(viral_peptides)

  0%|          | 0/195 [00:00<?, ?it/s]

10104498

In [6]:
# with open(hdf5_file, 'r+') as f:
#     dset4 = f.create_dataset('viral_peptides', data=list(viral_peptides))

In [17]:
# bacterial_peptides = set()
# collected_bacterial_peptides = set()

with h5py.File(hdf5_file, 'r+') as f:
    # vir_grp = f.create_group("viral_proteomes")
    for fname in tqdm(os.listdir("../data/UniProt/viral_proteomes")):
        if not fname.endswith("fasta.gz"):
            continue
        proteome_id = fname.split(".")[0]
        
        with gzip.open("../data/UniProt/viral_proteomes/{}".format(fname), "r") as f:
            viral_proteins = []
            prot_seq = ""
            # for line in tqdm(f):
            for line in f:
                ln = line.decode()
                if ln.startswith(">"):
                    if len(prot_seq)>0:
                        viral_proteins.append(prot_seq)
                        prot_seq = ""
                    continue
                else:
                    prot_seq += ln.strip()
            viral_proteins.append(prot_seq)

        species_peptides = []
        for l in range(MIN_PEPTIDE_LEN, MAX_PEPTIDE_LEN+1):
            for prot_seq in viral_proteins:
                peptides = extract_peptides_from_protein_sequence(prot_seq, l)
                species_peptides.extend(peptides)
        species_peptides = [p for p in species_peptides if p not in reference_human_peptides]
        species_peptides = list(set(species_peptides))
        # dset = vir_grp.create_dataset(proteome_id, data=species_peptides, chunks=True)
        print("Proteome {}, n.peptides = {}".format(proteome_id, len(species_peptides)))
        n_other_peptides += len(species_peptides)

  0%|          | 0/195 [00:00<?, ?it/s]

Proteome UP000127637, n.peptides = 270680
Proteome UP000297044, n.peptides = 10706
Proteome UP000113968, n.peptides = 272450
Proteome UP000007667, n.peptides = 11477
Proteome UP000007083, n.peptides = 181622
Proteome UP000171868, n.peptides = 49247
Proteome UP000214863, n.peptides = 172739
Proteome UP000130331, n.peptides = 10491
Proteome UP000002474, n.peptides = 16592
Proteome UP000002473, n.peptides = 16691
Proteome UP000244836, n.peptides = 11086
Proteome UP000000941, n.peptides = 168130
Proteome UP000009294, n.peptides = 187965
Proteome UP000153518, n.peptides = 10964
Proteome UP000297076, n.peptides = 10384
Proteome UP000464024, n.peptides = 49440
Proteome UP000102742, n.peptides = 15026
Proteome UP000009115, n.peptides = 11502
Proteome UP000163734, n.peptides = 12865
Proteome UP000159358, n.peptides = 170917
Proteome UP000677407, n.peptides = 163230
Proteome UP000149016, n.peptides = 183099
Proteome UP000009122, n.peptides = 12042
Proteome UP000103899, n.peptides = 307462
Proteo

In [18]:
n_other_peptides

790551832